## word2vec ##

In [ ]:
import json

from utils import *

In [ ]:
Chinesecorpus=open('../bishe_e1/data/mycorpus.txt','rt',encoding='utf-8').read()
cs_list = list(Chinesecorpus)
cs_set_list = list(set(cs_list))
cs_set_list.sort(key = cs_list.index)
corpus_char=''.join(cs_set_list)   
charset=ChineseCharset(corpus_char)            ### 2192 +'<UNK>'+<PAD>

In [ ]:
len(charset)

In [ ]:
with open('D:/词向量/totalcase_256d.txt','r',encoding='utf-8') as f:
    lines = f.readlines();
f.close()

In [ ]:
# count = 0
# for i in charset:
#     if i in w2v_vocab:
#         count += 1
# print(count/len(charset))   ## 0.9708295350957156     256词向量中覆盖的字数
 
# print(count,len(charset))   ## 2130 2194             前者是被覆盖的数，后者是字表长度 

In [ ]:
key2id = {}
id2vector = []
index = 0
for line in lines[1:]:
    line = line.strip()
    line = line.split(' ',1)
    id2vector.append(line[1].split(' '))
    key2id[line[0]] = index
    index += 1
#     w2v_vocab.append(line[0])
    

    

In [ ]:
len(key2id) == len(id2vector)

In [ ]:
len(key2id)

In [ ]:
import numpy as np

In [ ]:
count = 0
char_embedding = []
for i in charset:
    index = key2id.get(i,-1)
    if index == -1:
        vector = np.random.rand(256)
    else:
#         vector = id2vector[index]
        vector = np.array(id2vector[index],dtype='float32')
        count += 1
        
    char_embedding.append(vector)

In [ ]:
np.save('../data/char_embedding.npy',char_embedding)

In [ ]:
np.load('../data/char_embedding.npy')

### 开始训练 ###

In [1]:
from utils import *
from torch.utils.data.dataset import *
from torch.utils.data.sampler import *  #用于采样
from torch.nn.utils.rnn import *
import bisect
from model import *
import torch
import os
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time

torch.Size([2194, 256])


In [2]:
# parser = argparse.ArgumentParser(description="Joint Extraction of Entities and Relations")
batch_size = 32


dropout = 0.5

emb_dropout = 0.25

clip = 0.35

epochs = 3

char_kernel_size = 3

word_kernel_size = 3

emsize = 50

char_layers = 3

word_layers = 3

char_nhid = 50

word_nhid = 300

log_interval = 100

lr = 4

aoptim = 'SGD'

seed = 1111

save = 'model.pt'

weight = 10.0

In [3]:
cuda = False

In [4]:
# Set the random seed manually for reproducibility.
torch.manual_seed(seed)

device = torch.device("cuda" if cuda else "cpu")#torch.device代表将torch.Tensor分配到的设备的对象


In [5]:
Chinesecorpus=open('../bishe_e1/data/mycorpus.txt','rt',encoding='utf-8').read()
cs_list = list(Chinesecorpus)
cs_set_list = list(set(cs_list))
cs_set_list.sort(key = cs_list.index)
corpus_char=''.join(cs_set_list)   
charset=ChineseCharset(corpus_char)            ### 2192 +'<UNK>'+<PAD>

In [6]:
# charset = Charset()#定义字符对象
# vocab = Vocabulary()#定义词汇对象
# vocab.load("data/myvocab.txt")#读文件
tag_set = Index()#定义对象
tag_set.load("../data/my_tag2id.txt")#读文件  标记-id
relation_labels = Index()#定义对象
relation_labels.load('../data/my_relation_labels.txt')#读文件   关系标签

In [7]:
train_data = load('../data/my_train.pk')

val_size = int(0.05 * len(train_data))#设置验证集大小
train_data, val_data = random_split(train_data, [len(train_data)-val_size, val_size])##torch里的函数 随机切分训练集和验证集

In [8]:
len(train_data),len(val_data)

(2091, 110)

### 分组排序 ###

In [9]:
def group(data, breakpoints):# 把数据集按照句子长度分组#data:包含三部分 句子中每个词的id 句子中每个词对应的char的id 句子对应的实体id
    groups = [[] for _ in range(len(breakpoints)+1)]
    for idx, item in enumerate(data):
        i = bisect.bisect_left(breakpoints, len(item[0]))
        groups[i].append(idx)
    data_groups = [Subset(data, g) for g in groups]  #Subset是torch.utils.data.dataset中的
    return data_groups

In [10]:
train_data_groups = group(train_data, [50, 100, 150, 200, 250])
val_data_groups = group(val_data, [50, 100, 150, 200, 250])    ####一个分组排序的过程

In [11]:
print(train_data_groups[0][1])   

([1266, 402, 65, 21, 95, 124, 210, 202, 4, 5, 6, 19, 20, 21, 688, 23, 23, 335, 186, 336, 325, 43, 44, 45, 46, 47, 48, 49, 50, 51, 9, 10, 118, 8, 56, 56, 56, 119, 52], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 27, 29, 0, 0, 0, 0, 26, 28, 28, 28, 30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


### 采样与填充 ###

In [12]:
class GroupBatchRandomSampler(object):##每一组的句子长度不同 分别建采样器进行随机采样
    def __init__(self, data_groups, batch_size, drop_last):
        self.batch_indices = []
        for data_group in data_groups:
            self.batch_indices.extend(list(BatchSampler(SubsetRandomSampler(data_group.indices),
                                                        batch_size, drop_last=drop_last)))

    def __iter__(self):
        return (self.batch_indices[i] for i in torch.randperm(len(self.batch_indices)))

    def __len__(self):
        return len(self.batch_indices)

In [13]:
def get_batch(batch_indices, data):  #分batch   data有三部分   返回三部分的batch
    #print(batch_indices,len(data))
    batch = [data[idx] for idx in batch_indices]
    sorted_batch = sorted(batch, key=lambda x: len(x[0]), reverse=True)
    sentences, tags = zip(*sorted_batch)
#     print([len(s) for s in sentences])

    padded_sentences, lengths = pad_packed_sequence(pack_sequence([torch.LongTensor(_) for _ in sentences],enforce_sorted=False),  #文本长度对齐
                                                    batch_first=True, padding_value=charset["<pad>"])
    padded_tags, _ = pad_packed_sequence(pack_sequence([torch.LongTensor(_) for _ in tags],enforce_sorted=False),
                                         batch_first=True, padding_value=tag_set["O"])

    return padded_sentences.to(device),  padded_tags.to(device), lengths.to(device)

In [14]:
char_embedding = torch.Tensor(np.load('../data/char_embedding.npy'))
char_embedding_size = char_embedding.size(1)


# word_embeddings = torch.tensor(np.load("data/NYT_CoType/word2vec.vectors.npy"))      #将词向量转换为张量 47463*300
# word_embedding_size = word_embeddings.size(1)
# pad_embedding = torch.empty(1, word_embedding_size).uniform_(-0.5, 0.5)  #随机初始化 1*word_embedding_size 大小的张量 取值在-0.5~0.5之间
# unk_embedding = torch.empty(1, word_embedding_size).uniform_(-0.5, 0.5)
# word_embeddings = torch.cat([pad_embedding, unk_embedding, word_embeddings])

In [15]:
len(char_embedding) == len(charset)

True

In [16]:
batch_indices = [1,2,3,4,5]
sentences, targets, lengths = get_batch(batch_indices, train_data) #分batch

# sentences[0]

# charset[20],charset[21],charset[522],charset[23],charset[88]

# charset['鹰']

In [17]:
len(sentences[4]),len(targets[4])

(276, 276)

### 模型搭建 ###

In [18]:
char_channels = [char_embedding_size] + [char_nhid] * char_layers

In [19]:
sentences, targets, lengths = get_batch([0,1,2,3,4], train_data)
# out = Model(sentences)

In [20]:
if os.path.exists("model.pt"):      #是否存在已经构造好的模型
    model=torch.load('model.pt')
#——————————搭建模型————————————
else:
    model = Model( weight=char_embedding,
                  char_embedding_size=char_embedding_size, char_channels=char_channels,
                  char_kernel_size=char_kernel_size, num_tag=len(tag_set), dropout=dropout,
                  emb_dropout=emb_dropout).to(device)

In [21]:
out = model(sentences)

In [22]:
(out[0][0][0])

tensor(-4.1018, grad_fn=<SelectBackward>)

In [23]:
weight = [weight] * len(tag_set)   #生成一维向量 初始化值为arg.weight 维度为len(tag_set)
weight[tag_set["O"]] = 1     #"O"标签对应的权值为1


weight = torch.tensor(weight).to(device) 

In [24]:
criterion = nn.NLLLoss(weight, size_average=False)      #定义的损失函数
optimizer = getattr(optim, aoptim)(model.parameters(), lr=lr)  #getattr 获取torch.optim对象的属性值 默认是SGD(随机梯度下降)   得到优化器的损失函数(模型参数)


C:\Users\IR LAB\Anaconda3\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [25]:
def train():
    model.train()    #torch的模型训练
    total_loss = 0
    count = 0

    sampler = GroupBatchRandomSampler(train_data_groups, batch_size, drop_last=False)#采样器是定义的采样器对象 按组分batch分别采样

    for idx, batch_indices in enumerate(sampler):
        sentences, targets, lengths = get_batch(batch_indices, train_data) #分batch
    #     print(lengths)
        optimizer.zero_grad()
        output = model(sentences)  #词级别和字符级别建模
        output = pack_padded_sequence(output, lengths, batch_first=True,enforce_sorted=False).data #文本长度对齐
        targets = pack_padded_sequence(targets, lengths, batch_first=True,enforce_sorted=False).data #将标签的编码长度对齐
        loss = criterion(output, targets)  #损失函数
        loss.backward()
        if clip > 0:
            nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        total_loss += loss.item()
        count += len(targets)
#         print('Hello world!!!')
        if (idx+1) % log_interval == 0:
            cur_loss = total_loss / count
            elapsed = time.time() - start_time
            percent = ((epoch-1)*len(sampler)+(idx+1))/(epochs*len(sampler))
            remaining = elapsed / percent - elapsed
            print("| Epoch {:2d}/{:2d} | Batch {:5d}/{:5d} | Elapsed Time {:s} | Remaining Time {:s} | "
                  "lr {:4.2e} | Loss {:5.3f} |".format(epoch, epochs, idx+1, len(sampler), time_display(elapsed),
                                                       time_display(remaining), lr, cur_loss))
            total_loss = 0
            count = 0


In [26]:
def evaluate(data_groups):
    model.eval()
    total_loss = 0
    count = 0.000000001
    TP = 0.000000001
    TP_FP = 0.000000001
    TP_FN = 0.000000001
    with torch.no_grad():
        for batch_indices in GroupBatchRandomSampler(data_groups, batch_size, drop_last=False):
            sentences,targets, lengths = get_batch(batch_indices, train_data)
            output = model(sentences)
            tp, tp_fp, tp_fn = measure(output, targets, lengths,sentences)
            TP += tp
            TP_FP += tp_fp
            TP_FN += tp_fn
            output = pack_padded_sequence(output, lengths, batch_first=True).data
            targets = pack_padded_sequence(targets, lengths, batch_first=True).data
            loss = criterion(output, targets)
            total_loss += loss.item()
            count += len(targets)
            print(count,TP_FP,TP_FN)
    return total_loss / count, TP/TP_FP, TP/TP_FN, 2*TP/(TP_FP+TP_FN)

In [27]:
#measure(output, targets, lengths,sentences)
def measure(output, targets, lengths,sentences):
    assert output.size(0) == targets.size(0) and targets.size(0) == lengths.size(0)
    tp = 0
    tp_fp = 0
    tp_fn = 0
    batch_size = output.size(0)
    output = torch.argmax(output, dim=-1)
    for i in range(batch_size):
        sentence = sentences[i]
        length = lengths[i]
        out = output[i][:length].tolist()
        target = targets[i][:length].tolist()
        out_triplets = song_get_triplets(out,sentence)
        print("out:",out_triplets)
        tp_fp += len(out_triplets)
        target_triplets = song_get_triplets(target,sentence)
        print("target:",target_triplets)
        tp_fn += len(target_triplets)
        for target_triplet in target_triplets:
            for out_triplet in out_triplets:
                if out_triplet == target_triplet:
                    tp += 1
    return tp, tp_fp, tp_fn

In [28]:
def song_get_triplets(tags,outset):
    outsent = charset
#     outsent = out[0]
#     tags = out[1]
#     assert len(outsent) == len(tags)
    temp = {}
    triplets = []
    for idx, tag in enumerate(tags):
        if tag == tag_set["O"]:
            continue
    #     entity=[]
        pos, relation_label, role = tag_set[tag].split("-")
        if pos == "B" or pos == "S":
            entity=[]
            entity.append(idx)
        if pos=="I" or pos=="E":
            try:
                entity.append(idx)
            except UnboundLocalError:
                entity = []
                entity.append(idx)
        if pos=="S" or pos=="E":
            if relation_label not in temp:
                temp[relation_label] = [[], []]
            temp[relation_label][int(role) - 1].append(entity)
    try:
        label = list(temp.keys())[0]
    except IndexError:
        triplets.append([])
        triplets.append([])
        triplets.append([])
        return triplets
#     print(label)
    entitys = temp[label]
#     print('entitys:',entitys)

    entity_tmp = entitys[0]   
    if len(entity_tmp) == 0:
        entity1 = []
    else:
        entity1 = entity_tmp[0]
        
    entity_tmp = entitys[1]  
    if len(entity_tmp) == 0:
        entity2 = []
    else:
        entity2 = entity_tmp[0]


    entity1_word = []  
#     print(entity1)
    for i in entity1:
#         print(outset[i])
        entity1_word.append(outsent[int(outset[i])])
#     print(entity1_word)
    entity1_word = ''.join(entity1_word)
#     print(entity1_word)

    entity2_word = []
    for i in entity2:
        entity2_word.append(outsent[int(outset[i])])
#     print(entity2_word)
    entity2_word = ''.join(entity2_word)
#     print(entity2_word)
    
    triplets.append(entity1_word)
    triplets.append(label)
    triplets.append(entity2_word)
    
    
    return triplets

In [29]:
epochs = 3

In [30]:
best_val_loss = None
lr = lr
all_val_loss = []
all_precision = []
all_recall = []
all_f1 = []

# At any point you can hit Ctrl + C to break out of training early.
try:
    start_time = time.time()
    print("-" * 118)
    for epoch in range(1, epochs+1):
        train()
        val_loss, precision, recall, f1 = evaluate(val_data_groups) #评估的是验证集

        elapsed = time.time() - start_time
        print("-" * 118)
        print("| End of Epoch {:2d} | Elapsed Time {:s} | Validation Loss {:5.3f} | Precision {:5.3f} "
              "| Recall {:5.3f} | F1 {:5.3f} |".format(epoch, time_display(elapsed),
                                                       val_loss, precision, recall, f1))
        print("-" * 118)

        # Save the model if the validation loss is the best we've seen so far.
        if not best_val_loss or val_loss < best_val_loss:
            with open(save, 'wb') as f:
                torch.save(model, f)
            best_val_loss = val_loss
        else:
            # Anneal the learning rate if no improvement has been seen in the validation dataset.
            lr = lr / 4.0
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
        all_val_loss.append(val_loss)
        all_precision.append(precision)
        all_recall.append(recall)
        all_f1.append(f1)

except KeyboardInterrupt:
    print('-' * 118)
    print('Exiting from training early')


with open("record-batchsize64-epochs30-lr4.tsv", "wt", encoding="utf-8") as f:
    for idx in range(len(all_val_loss)):
        f.write("{:d}\t{:5.3f}\t{:5.3f}\t{:5.3f}\t{:5.3f}\n"
                .format(idx+1, all_val_loss[idx], all_precision[idx], all_recall[idx], all_f1[idx]))

----------------------------------------------------------------------------------------------------------------------
out: ['某某', 'provide_shelter_for', '某某']
target: ['员当场', 'posess', '条、吸']
out: ['翁某某', 'provide_shelter_for', '']
target: ['获的用', 'posess', '丙胺和']
out: ['罗××', 'provide_shelter_for', '黄××']
target: ['黄××', 'traffic_in', '海洛因']
out: ['李某某', 'provide_shelter_for', '陈某']
target: ['陈美燕', 'sell_drugs_to', '邹某某']
out: ['罗某', 'provide_shelter_for', '']
target: ['罗某', 'traffic_in', '冰毒']
out: ['刘某', 'provide_shelter_for', '刘某']
target: ['刘某', 'posess', '氯胺酮']
out: ['曾某', 'provide_shelter_for', '林某某']
target: ['曾某', 'provide_shelter_for', '林某某']
out: ['周某', 'provide_shelter_for', '周某基苯丙']
target: ['周某', 'traffic_in', '麻果']
out: ['王某', 'provide_shelter_for', '王某']
target: ['王某甲', 'provide_shelter_for', '王某']
out: ['谭某', 'provide_shelter_for', '唐某某']
target: ['谭某', 'provide_shelter_for', '廖某']
out: ['某', 'provide_shelter_for', '胡某']
target: ['丁某', 'traffic_in', '冰毒']
out: ['潘某某',

target: ['号门口', 'sell_drugs_to', '深圳']
out: ['叶某', 'provide_shelter_for', '王某']
target: ['法鉴', 'sell_drugs_to', '份。当']
out: ['肖某', 'provide_shelter_for', '张某']
target: ['肖某', 'sell_drugs_to', '张某丙']
out: ['朱某', 'provide_shelter_for', '朱某某']
target: ['朱某某', 'provide_shelter_for', '朱某1']
out: ['姚某某', 'provide_shelter_for', '姚某某']
target: ['在房间', 'provide_shelter_for', '在长沙']
out: ['章×××××', 'provide_shelter_for', '章×××××']
target: ['汪章胜', 'posess', '大麻叶']
out: ['邓某基', 'provide_shelter_for', '']
target: ['邓某', 'posess', '甲基苯丙胺']
out: ['曹某', 'provide_shelter_for', '罗某']
target: ['曹某', 'provide_shelter_for', '张某"4']
out: ['秦xxxx', 'provide_shelter_for', '秦xxxx']
target: ['杨某', 'posess', '甲基苯丙胺']
out: ['许某某XXX', 'provide_shelter_for', '许某某XXX']
target: ['钱某某', 'posess', '甲基苯丙胺']
out: ['李某某', 'provide_shelter_for', '']
target: ['李某某', 'posess', '甲基苯丙胺']
out: ['郑某某某', 'provide_shelter_for', '苏某某洛洛洛']
target: ['郑某', 'sell_drugs_to', '苏某某']
out: ['甘某', 'provide_shelter_for', '赵某']
target: ['甘某',

target: ['吴某', 'posess', '甲基苯丙胺']
out: ['郭某', 'provide_shelter_for', '']
target: ['郭某', 'provide_shelter_for', '刘某某']
15230.000000001 258.000000001 258.000000001
out: ['李某某', 'provide_shelter_for', '']
target: ['××又', 'sell_drugs_to', '家过夜']
out: ['胡忠', 'posess', '甲基苯丙胺']
target: ['胡修忠', 'traffic_in', '冰毒']
out: ['某', 'provide_shelter_for', '']
target: ['许某某', 'sell_drugs_to', '蒋某某']
out: ['', 'posess', '甲基苯丙胺']
target: ['大黄毛', 'traffic_in', '冰毒']
out: ['李某某', 'provide_shelter_for', '']
target: ['邓某某', 'posess', '甲基苯丙胺']
out: ['吴某', 'provide_shelter_for', '']
target: ['吴某某', 'traffic_in', '甲基苯丙胺']
out: ['某', 'provide_shelter_for', '']
target: ['莫某某', 'provide_shelter_for', '陈某']
out: ['某某', 'provide_shelter_for', '']
target: ['莫某某', 'posess', '甲基苯丙胺']
out: ['宋某', 'provide_shelter_for', '']
target: ['宋某某', 'provide_shelter_for', '李某']
out: ['王某', 'provide_shelter_for', '']
target: ['王某', 'sell_drugs_to', '符某某']
out: ['吉某某', 'provide_shelter_for', '']
target: ['吉地某某', 'sell_drugs_to', '桑

12059.000000001 210.000000001 210.000000001
out: ['李某某×', 'provide_shelter_for', '某某']
target: ['××又', 'sell_drugs_to', '家过夜']
out: ['胡忠', 'traffic_in', '']
target: ['胡修忠', 'traffic_in', '冰毒']
out: ['许某某', 'provide_shelter_for', '蒋某某']
target: ['许某某', 'sell_drugs_to', '蒋某某']
out: ['辛某某', 'provide_shelter_for', '']
target: ['大黄毛', 'traffic_in', '冰毒']
out: ['邓某某', 'provide_shelter_for', '李某']
target: ['邓某某', 'posess', '甲基苯丙胺']
out: ['吴某某', 'provide_shelter_for', '']
target: ['吴某某', 'traffic_in', '甲基苯丙胺']
out: ['莫某某', 'provide_shelter_for', '王某']
target: ['莫某某', 'posess', '甲基苯丙胺']
out: ['莫某某', 'provide_shelter_for', '王某']
target: ['莫某某', 'provide_shelter_for', '陈某']
out: ['宋某某某某某', 'provide_shelter_for', '宋某某某某某']
target: ['宋某某', 'provide_shelter_for', '李某']
out: ['王某', 'provide_shelter_for', '符某某']
target: ['王某', 'sell_drugs_to', '符某某']
out: ['吉地某某', 'provide_shelter_for', '孙某']
target: ['吉地某某', 'sell_drugs_to', '桑某']
out: ['徐某某', 'provide_shelter_for', '刘某']
target: ['徐某某', 'provide_she

In [ ]:
int(torch.tensor(28))

In [125]:
import pickle

with open('../data/my_train.pk', 'rb') as f1:
    train_data = pickle.load(f1)
f1.close()

In [127]:
with open('../data/my_train.txt','r',encoding='utf-8') as f:
    lines = f.readlines()
f.close()

In [128]:
tags = train_data[0][1]

outsent = list(lines[0].strip())

In [142]:
sentences = train_data[0][0]

In [129]:
### 处理tagset
tag_set = Index()#定义对象
tag_set.load("../data/my_tag2id.txt")#读文件  标记-id

### 处理relation
relation_labels = Index()#定义对象
relation_labels.load('../data/my_relation_labels.txt')#读文件   关系标签

In [143]:
song_get_triplets(tags,sentences)

entity1:  [25, 26, 27]
['简', '某', '某']
['简', '某', '某']


['简某某', 'provide_shelter_for', '简某某']

In [141]:
outsent[2]

'机'

In [137]:
print(outsent)

['公', '诉', '机', '关', '指', '控', '，', '2', '0', '1', '5', '年', '1', '1', '月', '份', '的', '一', '天', '晚', '上', '，', '被', '告', '人', '简', '某', '某', '容', '留', '陈', '某', '、', '吴', '某', '、', '杨', '某', '某', '、', '张', '某', '、', '伍', '某', '某', '在', '简', '某', '某', '家', '一', '楼', '卧', '室', '，', '以', '烫', '吸', '的', '方', '式', '吸', '食', '甲', '基', '苯', '丙', '胺', '（', '冰', '毒', '）', '。']


In [136]:
print(tags)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 4, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [41]:
''.join([])

''

In [48]:
charset['我喜欢你']

2193

In [45]:
# class CharEncoder(nn.Module):
#     """
#     Input:  (batch_size, seq_len, char_features)
#     """
#     def __init__(self, weight, channels, kernel_size, dropout, emb_dropout):  #weight 是word_embedding
#         super(CharEncoder, self).__init__()
#         self.embed = nn.Embedding.from_pretrained(weight, freeze=False)    ##加载词向量
#         self.drop = nn.Dropout(emb_dropout)
#         self.conv_net = ConvNet(channels, kernel_size, dropout, dilated=True, residual=False)
#         #print(weight.size(0),weight.size(1))
#     def forward(self,  char_input):
#         # (batch_size, seq_len) -> (batch_size, seq_len, embedding_size)
#         #  -> (batch_size, seq_len, embedding_size + char_features)
#         #  -> (batch_size, embedding_size + char_features, seq_len)
#         embeddings=self.embed(char_input)
# #         embeddings = torch.cat((embeddings, char_input), 2)                     ### 将字与词连接起来，以第2维度连接起来
#         embeddings=embeddings.transpose(1, 2).contiguous()                      ### 矩阵再次转置

#         #print("embeddings:----------",embeddings.size())

#         # (batch_size, embedding_size + char_features, seq_len) -> (batch_size, conv_size, seq_len)
#         conv_out = self.conv_net(self.drop(embeddings))

#         # torch.cat(embeddings, conv_out), 1) ==>(batch_size, embedding_size + char_features, seq_len) -> (batch_size, conv_size + embedding_size + char_features, seq_len)
#         #  -> (batch_size, seq_len, conv_size + embedding_size + char_features)
#         return torch.cat((embeddings, conv_out), 1).transpose(1, 2).contiguous()

# char_encoder = CharEncoder(char_embedding, word_channels, word_kernel_size,dropout=dropout, emb_dropout=emb_dropout)

# sentences, targets, lengths = get_batch([0,1,2,3,4], train_data)

# char_output = char_encoder(sentences)

# len(char_output[0])  276

# class Decoder(nn.Module):
#     def __init__(self,input_size,hidden_dim,output_size,NUM_LAYERS):
#         super(Decoder, self).__init__()
#         self.input_size=input_size
#         self.hidden_dim = hidden_dim      ###有
#         self.output_size=output_size      ### 分为多少类
#         self.lstm = nn.LSTM(input_size, hidden_dim, num_layers = NUM_LAYERS)
#         self.hidden2label = nn.Linear(hidden_dim, output_size)
#         self.init_weight()

#     def forward(self, inputs):
#         self.lstm.flatten_parameters()
#         #print(self.hidden_dim,self.input_size,self.output_size)
#         # self.hidden = self.init_hidden(inputs.size(1))
#         self.hidden = self.init_hidden(inputs.size(1))
#         lstm_out, self.hidden = self.lstm(inputs,self.hidden)
#         #lstm_out, self.hidden = self.lstm(inputs,None)
#         y = self.hidden2label(lstm_out)
#         return y

#     def init_weight(self):
#         nn.init.kaiming_uniform_(self.hidden2label.weight.data, mode='fan_in', nonlinearity='relu')

#     def init_hidden(self, batch_size):
#         return (autograd.Variable(torch.randn(1, batch_size, self.hidden_dim)),
#                 autograd.Variable(torch.randn(1, batch_size, self.hidden_dim)))

# class Model(nn.Module):
#     def __init__(self,
#                  weight, char_embedding_size, char_channels, char_kernel_size, num_tag, dropout, emb_dropout):    ##搭建网络结构
#         super(Model, self).__init__()

#         self.char_encoder = CharEncoder(char_embedding, char_channels, char_kernel_size,dropout=dropout, emb_dropout=emb_dropout)
#         self.drop = nn.Dropout(dropout)
# #         self.char_conv_size = char_channels[-1]
#         self.char_embedding_size = char_embedding_size
#         self.char_conv_size = char_channels[-1]
#         #self.decoder = nn.Linear(self.char_conv_size+self.word_embedding_size+self.word_conv_size, num_tag)
#         self.decoder = Decoder(self.char_embedding_size+self.char_conv_size,    ###输入特征
#                                64 + self.char_embedding_size + self.char_conv_size,###隐藏层维度
#                                num_tag,NUM_LAYERS=1)

#         self.init_weights()

#     def forward(self, char_input):
#         batch_size = char_input.size(0)  #32
#         seq_len = char_input.size(1)    #句子长度
# #         char_input=char_input.contiguous()
# #         #print(char_input.view(-1, char_input.size(2)).size(0),char_input.view(-1, char_input.size(2)).size(1))##3200*10
# #         char_output = self.char_encoder(char_input.view(-1, char_input.size(2))).view(batch_size, seq_len, -1)#char_input.size(2)==20  调用forward方法
#         char_output = self.char_encoder(char_input)  ##(batch_size, seq_len, conv_size + embedding_size + char_features)
#         y = self.decoder(char_output)

#         return F.log_softmax(y, dim=2)
# #         return F.log_softmax(y, dim=1)

#     def init_weights(self):
#         pass

# word_embeddings = torch.tensor(np.load("../data/char_embedding.npy"))